In [49]:
'''

DNN, CNN, RNN
embedding: tfid, word2vec

cnn architecture
embedding - 300 dim
max_len = 40

conv2d = None,40,64
conv2d = None,40,32
maxpool = None,13,32


conv1d = None,13,16
conv1d = None, 13,8
global avg pool = None,18
Dense = None,1





'''


'\n\nDNN, CNN, RNN\nembedding: tfid, word2vec\n\ncnn architecture\nembedding - 300 dim\nmax_len = 40\n\nconv2d = None,40,64\nconv2d = None,40,32\nmaxpool = None,13,32\n\n\nconv1d = None,13,16\nconv1d = None, 13,8\nglobal avg pool = None,18\nDense = None,1\n\n\n\n\n\n'

In [50]:
!pip install kaggle

In [51]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [52]:
# api to fetch dataset
!kaggle datasets download -d kazanova/sentiment140

sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [53]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('Dataset extracted')

Dataset extracted


In [54]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [55]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Data processing

In [56]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding = 'ISO-8859-1')

In [57]:
# checking no. of rows and cols
twitter_data.shape

(1599999, 6)

In [58]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [59]:
# naming the columns and reading the dataset again

column_names = ['target', 'id', 'data', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = column_names, encoding = 'ISO-8859-1')

In [60]:
twitter_data.shape

(1600000, 6)

In [61]:
twitter_data.head ()

,target,id,data,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [62]:
twitter_data.isnull().sum()

target    0
id        0
data      0
flag      0
user      0
text      0
dtype: int64

In [63]:
# checking the distribution of target column
twitter_data['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [64]:
# converting target = 4 to 1
twitter_data.replace({'target':{4:1}},inplace=True)

In [65]:
twitter_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

# stemming
reducin the word to its root word

In [66]:
port_stem = PorterStemmer()

In [67]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [68]:
# taking randomly 16000 rows from dataset
# twitter_data = twitter_data.sample(n=16000, random_state=42)

In [69]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [70]:
twitter_data.head()

,target,id,data,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [91]:
twitter_data[['stemmed_content', 'target']].to_csv('twitter_data.csv', index=False)

In [71]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [72]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [73]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [74]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [75]:
print(Y)

[0 0 0 ... 1 1 1]


# splitting the data into training and test data

In [76]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=5,stratify=Y)

In [77]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [78]:
print(X_test)

['emili like pee deep end ahhaah'
 'mean see aaaaaaaaand way dailybooth meet best friend http tinyurl com mwsm w'
 'go ring shop ladi' ... 'nfrussel yeah road trip'
 'ptinsley damn suck load librari yet test music command ruin day'
 'mileycyru new song come along good hope hope well famili god bless chanc repli']


# converting textual data to numerical data

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train = vectorizer.fit_transform(X_train)

# Only transform the test data using the same vocabulary as learned from the training data
X_test = vectorizer.transform(X_test)

In [80]:
print(X_train[1])

  (0, 429569)	0.36069515294472226
  (0, 33752)	0.36069515294472226
  (0, 438493)	0.17920320711015286
  (0, 249196)	0.11625131115897651
  (0, 272655)	0.19223283926899112
  (0, 426531)	0.1476075287094744
  (0, 66382)	0.25248580745437876
  (0, 39493)	0.1402323392193017
  (0, 331003)	0.5650798659991658
  (0, 181713)	0.37735485447514105
  (0, 283017)	0.30050337168173064


In [81]:
print(X_test)

  (0, 311854)	0.4171144373494487
  (0, 234671)	0.18241073412253994
  (0, 120039)	0.27350243321462087
  (0, 118629)	0.39959613334366184
  (0, 96204)	0.3883553643040904
  (0, 6647)	0.6383113115794222
  (1, 436990)	0.21823309985776707
  (1, 409728)	0.272543871815237
  (1, 358217)	0.18851425508300926
  (1, 259198)	0.25961146603053226
  (1, 258502)	0.2499920169561539
  (1, 171195)	0.1742967177513577
  (1, 138886)	0.21710676644564525
  (1, 88887)	0.4596247711759963
  (1, 78763)	0.1870233790393096
  (1, 39493)	0.23582896297221057
  (1, 98)	0.5773211737315742
  (2, 365683)	0.49569025884239465
  (2, 341209)	0.6190504487881319
  (2, 225636)	0.5431230906491545
  (2, 149798)	0.27583512747234085
  (3, 324595)	0.20184426143574097
  (3, 319447)	0.2638179054473202
  (3, 283716)	0.2656885207337087
  (3, 261875)	0.3446914451420542
  :	:
  (319997, 342468)	0.6622239623228642
  (319998, 454554)	0.22177467425039196
  (319998, 399548)	0.26065775606923364
  (319998, 387191)	0.2159939663411723
  (319998, 3474

# training the machine learning model

In [82]:
model = LogisticRegression()

model.fit(X_train,Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [83]:
# Predict the labels for the test data
predictions = model.predict(X_test)

In [84]:
# Evaluate the model's performance
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.777590625


# saving the trained model

In [85]:
import pickle

In [86]:
filename = 'trained_model_twitter_sentiment.sav'
pickle.dump(model,open(filename,'wb'))  # writing in binary format

# using saved model for future predictions

In [87]:
model = pickle.load(open('/content/trained_model_twitter_sentiment.sav','rb'))

In [89]:
import random

X_new = X_test[200]

In [90]:
predictions = model.predict(X_new)

print(predictions)

if predictions[0] == 0:
  print('Negative Tweet')
else:
  print('Positive Tweet')

[1]
Positive Tweet


,stemmed_content,target
0,switchfoot http twitpic com zl awww bummer sho...,0
1,upset updat facebook text might cri result sch...,0
2,kenichan dive mani time ball manag save rest g...,0
3,whole bodi feel itchi like fire,0
4,nationwideclass behav mad see,0
